# Capstone Project - The Battle of Neighborhoods (Week 1)

In that project we are comparing neighborhoods of New York City and the city of Toronto by using Foursquare location data.

##  1. Introduction to the problem

We have a senerio for open a shop/restaurant in a city. 
Firstly ,we are focusing to analyze common and different venues in downtown Toronto and the western central London to understand their culture and lifestyle. Then we can come up with an idea for the right area for opening a new place.

## 2. Data

We will use the datasets below while starting to collect the data.

<n>The postal codes of western central London : https://en.wikipedia.org/wiki/WC_postcode_area     
<n>The postal codes of downtown Toronto : https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M     
<n>The geographical coordinates : Python Geocoder package     
<n>The types and locations of venues : Foursquare API     

## Gathering Data

In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

### London Data

In [ ]:
# Retrive London data from Wikipedia
sourcelondon = requests.get('https://en.wikipedia.org/wiki/WC_postcode_area').text
souplondon = BeautifulSoup(sourcelondon,'lxml')

tablelondon = soup1.find('table',{'class':'wikitable sortable'})

In [ ]:
# Loop through the rows to get the data
PostalCode =[]
PostTown = []
Neighbourhood = []

for row in table1.findAll("tr"):
    cells = row.findAll("th")
    if len(cells) == 1:
        PostalCode.append(cells[0].find(text=True))
    
    cells = row.findAll("td")
    if len(cells) == 3: 
        PostTown.append(cells[0].find(text=True))
        Neighbourhood.append(cells[1].find(text=True))

london = pd.DataFrame(PostalCode, columns = ['PostalCode'])
london['PostTown'] = PostTown
london['Neighbourhood'] = Neighbourhood
london.head()

#### Adding latitudes and longitudes

In [7]:
from geopy.geocoders import Nominatim
Latitude = []
Longitude = []

for i in london['Neighbourhood']:
    geolocator = Nominatim(user_agent="ld_explorer")
    location = geolocator.geocode(i)
    
    latitude = location.latitude
    Latitude.append(latitude)
    
    longitude = location.longitude
    Longitude.append(longitude)
    
london['Latitude'] = Latitude
london['Longitude'] = Longitude
london.head()

In [ ]:
# Droping far spots away from the neighbourhoods data
london = london.drop(london.index[3])
london = london.drop(london.index[11])
london.head()

### Toronto Data

In [ ]:
# Retrive Toronto data from Wikipedia
sourcetoronto = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
souptoronto = BeautifulSoup(sourcetoronto,'lxml')

tabletoronto = soup2.find('table',{'class':'wikitable sortable'})

In [ ]:
# Loop through the rows to get the data
PostalCode =[]
Borough = []
Neighbourhood =[]

for row in tabletoronto.findAll("tr"):
    cells = row.findAll("td")
    if len(cells) == 3:
        PostalCode.append(cells[0].find(text=True))
        Borough.append(cells[1].find(text=True))
        Neighbourhood.append(cells[2].find(text=True))
        
toronto = pd.DataFrame(PostalCode, columns = ['PostalCode'])
toronto['Borough'] = Borough
toronto['Neighbourhood'] = Neighbourhood
toronto.head()

### Cleaning Data of Toronto

In [ ]:
# Removing 'Not assigned' data in borough column
condition = toronto.Borough == 'Not assigned'
toronto = toronto.drop(toronto[condition].index, axis = 0, inplace = False)

In [ ]:
# Changing 'Not assigned' data in neighborhood column and replace it with borough data
toronto['Neighbourhood'] = toronto['Neighbourhood'].str.strip()

import numpy as np
toronto['Neighbourhood'] = np.where(toronto['Neighbourhood'] =='Not assigned', toronto['Borough'], toronto['Neighbourhood'])

In [ ]:
# Combining Neighbourhood column by using ',' when they have the same postal code
toronto2 = pd.DataFrame(toronto.groupby(['PostalCode','Borough'], as_index = False).agg(', '.join))
toronto2.head()

In [ ]:
geodata = pd.read_csv('https://cocl.us/Geospatial_data')

toronto3 = pd.concat([toronto2, geodata], axis=1).drop('Postal Code',axis = 1)
toronto3.head()

#We are using only downtown in Toronto
dt_trt = toronto3[toronto3['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dt_trt.head()

Let's look shortly to our datasets

In [ ]:
london.head()

In [ ]:
dt_trt.head()

## 3. Methodology

For both city:
We will visualize data in map by using folium map library.
Then we will look at 100 venues in each neighbourhood for around 500 mt by using Foursquare API.
Then we will machine learning algorithm called K Means Clustering for deviding neighbourhoods to 3 clusters.
In summary section, we will compare this two results for come up with ideas.

### Visualization

#### London

In [ ]:
address = 'London'

geolocator = Nominatim(user_agent="ld_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

In [ ]:
import folium

# Create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=12)

# Add markers to map
for lat, lng, borough, neighborhood in zip(london['Latitude'], london['Longitude'], london['PostTown'], london['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

Foursquare API

In [ ]:
CLIENT_ID = 'KULIKOK1JSONEBUQ1W0PCIR3DLSACJZSVOGJA3ZSIA1EEP24'
CLIENT_SECRET = '4P32UWQ5GUBPQFZ3YJZRUMKNBWWLKKGRLLPWFDW5QOH41KSZ'
VERSION = '20190120'

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
london_venues = getNearbyVenues(names = london['Neighbourhood'],
                                latitudes = london['Latitude'],
                                longitudes = london['Longitude'])
london_venues.head()

In [ ]:
# How many venues were returned for each neighborhood
london_venues.groupby('Neighbourhood').count()

In [ ]:
print('There are {} uniques categories of all the returned venues.'.format(len(london_venues['Venue Category'].unique())))

Top 10 most common venues for each group

In [ ]:
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighbourhood'] = london_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

# average frequency for each neighborhood group
london_grouped = london_onehot.groupby('Neighbourhood').mean().reset_index()
london_grouped.head()


In [ ]:
# Sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
london_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
london_neighborhoods_venues_sorted['Neighbourhood'] = london_grouped['Neighbourhood']

for ind in np.arange(london_grouped.shape[0]):
    london_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

london_neighborhoods_venues_sorted.head()

#### Clustering

In [ ]:
from sklearn.cluster import KMeans

london_grouped_clustering = london_grouped.drop('Neighbourhood', 1)

kclusters = 3
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(london_grouped_clustering)

kmeans.labels_[0:10] # cluster labels generated for each row in the dataframe


# add clustering labels
london_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = london
london_merged = london_merged.join(london_neighborhoods_venues_sorted.set_index('Neighbourhood'), 
                                   on ='Neighbourhood')

london_merged.head()

#### Visualization

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# create map
london_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'],london_merged['Longitude'],london_merged['Neighbourhood'],london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(london_clusters)
       
london_clusters

#### Labeling

In [ ]:
#theater
london_merged.loc[london_merged['Cluster Labels'] == 0, 
                  london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]
#cafe
london_merged.loc[london_merged['Cluster Labels'] == 1, 
                  london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]
#pub
london_merged.loc[london_merged['Cluster Labels'] == 2, 
                  london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

#### Toronto

In [ ]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# Create map of Downtown Toronto using latitude and longitude values
map_dt_trt = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(dt_trt['Latitude'], dt_trt['Longitude'], dt_trt['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dt_trt)  
    
map_dt_trt

Foursquare API

In [ ]:
dt_venues = getNearbyVenues(names = dt_trt['Neighbourhood'],
                            latitudes = dt_trt['Latitude'],
                            longitudes = dt_trt['Longitude'])

In [ ]:
# How many venues were returned for each neighborhood
dt_venues.groupby('Neighbourhood').count()

In [ ]:
print('There are {} uniques categories of all the returned venues.'.format(len(dt_venues['Venue Category'].unique())))

Top 10 most common venues for each group

In [ ]:
dt_onehot = pd.get_dummies(dt_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dt_onehot['Neighbourhood'] = dt_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [dt_onehot.columns[-1]] + list(dt_onehot.columns[:-1])
dt_onehot = dt_onehot[fixed_columns]

# average frequency for each neighborhood group
dt_grouped = dt_onehot.groupby('Neighbourhood').mean().reset_index()
dt_grouped.head()

In [ ]:
num_top_venues = 5

for hood in dt_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = dt_grouped[dt_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
# Sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
trt_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
trt_neighborhoods_venues_sorted['Neighbourhood'] = dt_grouped['Neighbourhood']

for ind in np.arange(dt_grouped.shape[0]):
    trt_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dt_grouped.iloc[ind, :], num_top_venues)

trt_neighborhoods_venues_sorted.head()

#### Clustering

In [ ]:
dt_grouped_clustering = dt_grouped.drop('Neighbourhood', 1)

kclusters = 6
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(dt_grouped_clustering)

kmeans.labels_[0:10] # cluster labels generated for each row in the dataframe

In [ ]:
# add clustering labels
trt_neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

dt_merged = dt_trt

# merge dt_grouped with dt_trt to add latitude/longitude for each neighborhood
dt_merged = dt_merged.join(trt_neighborhoods_venues_sorted.set_index('Neighbourhood'), 
                           on ='Neighbourhood')

dt_merged.head()

#### Visualization

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dt_merged['Latitude'], dt_merged['Longitude'], dt_merged['Neighbourhood'], dt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Labeling

In [ ]:
#cafe restaurant
dt_merged.loc[dt_merged['Cluster Labels'] == 0, 
              dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]
#park
dt_merged.loc[dt_merged['Cluster Labels'] == 1, 
              dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]
#grocery
dt_merged.loc[dt_merged['Cluster Labels'] == 2, 
              dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]
#airport
dt_merged.loc[dt_merged['Cluster Labels'] == 3, 
              dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]
#cafe
dt_merged.loc[dt_merged['Cluster Labels'] == 4, 
              dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]
#bar              
dt_merged.loc[dt_merged['Cluster Labels'] == 5, 
              dt_merged.columns[[1] + list(range(5, dt_merged.shape[1]))]]

## 4. Summary

In [ ]:
#Summary table for Toronto vs London spot grouping in the city
Clusters = [1,2,3,4,5]
comparison = pd.DataFrame(Clusters, columns = ['Clusters'])

London = ['Theater','Cafe','Pub','-','-']
Toronto = ['Cafe & Restaurant','Cafe','Bar','Park',' Grocery Store']

comparison['London'] = London
comparison['Toronto'] = Toronto

comparison

## 5. Discussion

It seems in the results that London and Toronto are very similar structure. We already know that both two city are multicultural and have wide variety for cafe nad restaurants.

## 6. Conclusion

Also both cities have differences. London has lots of green areas and parks. Start a fastfood truck business can be a good option.  